In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from models.reader import data_reader
import config
from utils.data_utils import dump_pkl, write_vocab, load_pkl, build_vocab, load_vocab
import numpy as np

In [ ]:
col_sep=config.col_sep
data_path=config.train_seg_path
min_count=config.min_count

In [ ]:
data_content, data_lbl = data_reader(data_path, col_sep)
word_lst = []
for i in data_content:
    word_lst.extend(i.split())

# word vocab
word_vocab = build_vocab(word_lst, min_count=min_count, sort=True, lower=True)

In [ ]:
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2),vocabulary=word_vocab, sublinear_tf=True)
data_feature = vectorizer.fit_transform(data_content)

In [ ]:
idf_featu = vectorizer.get_feature_names()

In [ ]:
len(data_content)

In [ ]:
len(idf_featu)

In [ ]:
import pickle as pkl
import codecs

In [ ]:
with codecs.open("/ssd1/zhubenchang/output/feature_tf_word.pkl", "rb") as f:
    vect = pkl.load(f)

In [ ]:
res = vect.fit_transform(data_content[:10000])

In [ ]:
with codecs.open("/ssd1/zhubenchang/output/model_tf_word_logistic_regression.pkl", "rb") as f:
    lr_model = pkl.load(f)

In [ ]:
arr = lr_model.predict(res)

In [ ]:
data_lbl = np.array(data_lbl)
data_content = np.array(data_content)

In [ ]:
data_content_yl = data_content[data_lbl=="2201"]

In [ ]:
data_content_yl

In [ ]:
res = vectorizer.fit_transform(data_content)
#pre_res = lr_model.predict(res)

In [ ]:
len(data_content)

In [ ]:
whole = data_feature.toarray()
top = whole[:5]

In [ ]:
len(top[1])

In [ ]:
features = vect.get_feature_names()

In [ ]:
len(features)


In [ ]:
data_feature_arr = data_feature.toarray()

In [ ]:
data_lbl = np.array(data_lbl)
data_content = np.array(data_content)

In [ ]:
key_top = []

In [ ]:
for ele in data_feature_arr[data_lbl=="2201"]:
    ele_sort = sorted(zip(idf_featu, ele), key = lambda x:x[1], reverse = True)
    print(ele_sort[:20])
    key_top.append(ele_sort[:20])
    

In [ ]:
print(data_content_yl[:5])

In [ ]:
data_content_yl[pre_res!=16]

In [ ]:
key_top[1]

In [ ]:
weights = lr_model.coef_
weight = weights[16]

In [ ]:
rank = sorted(zip(vect.get_feature_names(), weight), key = lambda x:x[1], reverse = True)

In [ ]:
rank[:10]

In [ ]:
import sys
sys.getsizeof(vect)/(1024**2)

## simhash

In [ ]:
import jieba
import jieba.analyse
import numpy as np
import json
 
class simhash:
    def __init__(self,content):
        self.simhash=self.simhash(content)
 
    def __str__(self):
        return str(self.simhash)
 
    def simhash(self,content):
        #seg = jieba.cut(content)
        #jieba.analyse.set_stop_words('data/stop_words.txt_utf8')
        #keyWord = jieba.analyse.extract_tags(
        #    '|'.join(seg), topK=20, withWeight=True, allowPOS=())#在这里对jieba的tfidf.py进行了修改
        #将tags = sorted(freq.items(), key=itemgetter(1), reverse=True)修改成tags = sorted(freq.items(), key=itemgetter(1,0), reverse=True)
        #即先按照权重排序，再按照词排序
        keyWord = content
        keyList = []
        # print(keyWord)
        for feature, weight in keyWord:
            #print(feature + str(weight))
            weight = int(weight * 20)
            feature = self.string_hash(feature)
            temp = []
            for i in feature:
                if(i == '1'):
                    temp.append(weight)
                else:
                    temp.append(-weight)
            # print(temp)
            keyList.append(temp)
        list1 = np.sum(np.array(keyList), axis=0)
        #print(list1)
        if(keyList==[]): #编码读不出来
            return '00'   
        simhash = ''
        for i in list1:
            if(i > 0):
                simhash = simhash + '1'
            else:
                simhash = simhash + '0'
        return simhash
 
 
    def string_hash(self,source):
        if source == "":
            return 0
        else:
            x = hash(source)
            x = bin(x).replace('0b', '').zfill(128)[-128:]
            #print(source,x)
 
            return str(x)
#     def string_hash(self,source):
#         if source == "":
#             return 0
#         else:
#             x = ord(source[0]) << 7
#             m = 1000003
#             mask = 2 ** 128 - 1
#             for c in source:
#                 x = ((x * m) ^ ord(c)) & mask
#             x ^= len(source)
#             if x == -1:
#                 x = -2
#             x = bin(x).replace('0b', '').zfill(64)[-64:]
#             #print(source,x)
 
#             return str(x)
 
        '''
        以下是使用系统自带hash生成，虽然每次相同的会生成的一样，
        不过，对于不同的汉子产生的二进制，在计算海明码的距离会不一样，
        即每次产生的海明距离不一致
        所以不建议使用。
        '''
        # x=str(bin(hash(source)).replace('0b','').replace('-','').zfill(64)[-64:])
        # print(source,x,len(x))
        # return x
 
 
    def hammingDis(self,com):
        t1 = '0b' + self.simhash
        t2 = '0b' + com.simhash
        n=int(t1, 2) ^ int(t2, 2)
        i=0
        while n:
            n &= (n-1)
            i+=1
        return i
 


In [ ]:
hash("什么")

In [ ]:
key_top_1104 = []
for ele in data_feature_arr[(data_lbl=="2201")|(data_lbl=="1104")]:
    ele_sort = sorted(zip(idf_featu, ele), key = lambda x:x[1], reverse = True)
    #print(ele_sort[:20])
    key_top_1104.append(ele_sort[:20])

In [ ]:
key_top_22_1104[6]

In [ ]:
com = simhash(key_top_22_1104[6])

In [ ]:
count = 0
acc_count = 0
diff_list = []
for text in key_top_22_1104:
    count += 1
    test_hash = simhash(text)
    diff = test_hash.hammingDis(com)
    #print(text)
    #print(diff)
    diff_list.append(diff)
    acc_count += 1
print(acc_count/float(count))

In [ ]:
len(data_lbl[(data_lbl=="2201") | (data_lbl=="1104")])

In [ ]:
count = 0
for ele in zip(diff_list, data_lbl[(data_lbl=="2201") | (data_lbl=="1104")]):
    if ele[1] == "2201":
        print(ele)
        count += 1
print(count)

In [ ]:
count = 0
for ele in zip(diff_list, data_lbl[(data_lbl=="2201") | (data_lbl=="1104")]):
    if ele[0] <= 25:
        print(ele)
        count += 1
print(count)

In [ ]:
15/18

In [ ]:
15/16

In [ ]:
for ele in zip(diff_list, data_lbl[(data_lbl=="2201") | (data_lbl=="1104")]):
    if ele[1] == "1104":
        print(ele)

## TFIDF + jecard

In [ ]:
key_top_22_1104 = []
for ele in data_feature_arr[(data_lbl=="2201")|(data_lbl=="1104")]:
    ele_sort = sorted(zip(idf_featu, ele), key = lambda x:x[1], reverse = True)
    #print(ele_sort[:20])
    key_top_22_1104.append(ele_sort[:20])

In [ ]:
sentence1 = np.array(key_top_22_1104[6]).T[0]

In [ ]:
def compute_jaccard_similarity(sentence1, sentence2):
    word_set1 = set(sentence1)
    word_set2 = set(sentence2)

    return len(word_set1 & word_set2) / len(word_set1 | word_set2)

In [ ]:
sim_list = []
for sentence in key_top_22_1104:
    sim = compute_jaccard_similarity(sentence1, np.array(sentence).T[0])
    sim_list.append(sim)

In [ ]:
for ele in zip(sim_list, data_lbl[(data_lbl=="2201") | (data_lbl=="1104")]):
    if ele[1] == "2201":
        print(ele)

In [ ]:
count = 0
for ele in zip(sim_list, data_lbl[(data_lbl=="2201") | (data_lbl=="1104")]):
    if ele[0] > 0.08:
        print(ele)
        count += 1
print(count)

In [ ]:
17/18

##  TFIDF+JECARD 评估

## LDA

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
lda = LatentDirichletAllocation(n_components=3, learning_method='batch', max_iter=25, random_state=0)
lda_topics = lda.fit_transform(data_feature_arr[(data_lbl=="2201") | (data_lbl=="1104")])

In [ ]:
lda.components_

In [ ]:
lda_topics

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    #打印每个主题下权重较高的term
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

    #打印主题-词语分布矩阵
    print("#主题-词语分布矩阵: \n %s" %model.components_)

n_top_words=20
print_top_words(lda, idf_featu, n_top_words)

In [ ]:
lda_label = []
for ele in lda_topics:
    lda_label.append(np.array(ele).argsort()[::-1][0])
    

In [ ]:
for ele in zip(lda_label, data_lbl[(data_lbl=="2201") | (data_lbl=="1104")]):
    print(ele)

##  LDA准召 100%

In [ ]:
from sklearn.cluster import SpectralClustering
import numpy as np
import math

X = np.array([[185.4, 72.6],
    [155.0, 54.4],
    [170.2, 99.9],
    [172.2, 97.3],
    [157.5, 59.0],
    [190.5, 81.6],
    [188.0, 77.1],
    [167.6, 97.3],
    [172.7, 93.3],
    [154.9, 59.0]])

w, h = 10, 10;

#构建相似度矩阵，任意两个样本间的相似度= 100 - 两个样本的欧氏距离
Matrix = [[100- math.hypot(X[x][0]- X[y][0], X[x][1]- X[y][1]) for x in range(w)] for y in range(h)]

sc = SpectralClustering(3, affinity='precomputed', n_init=10)
sc.fit(Matrix)

print('spectral clustering')   
print(sc.labels_)
print(sc.affinity_matrix_)

In [ ]:
a = [1]
b = [5]

In [ ]:
a += b

In [ ]:
a